# 遥感远程控制
在本例中，我们将使用连接到web浏览器机器的gamepad控制器远程控制Jetbot

### 创建游戏板控制器

我们要做的第一件事是创建一个 ``Controller`` 小部件的实例，我们将使用它来控制小车。

``Controller`` 小部件采用 ``index`` 参数来指定控制器的编号。如果连接多个控制器或者一些游戏板 *显示* 为多个控制器的时候，透过index确认使用的控制器会非常有用。
1. 访问[http://html5gamepad.com](http://html5gamepad.com) 。
2. 按下你正在使用的gamepad的按钮
3. 记住对按钮按下做出响应的游戏板的“索引”

接下来，我们将使用该索引创建并显示控制器。

In [ ]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=1)  # replace with index of your controller

display(controller)

即使索引正确，您也可能看到文本 ``Connect gamepad and press any button``。那是因为游戏板还没在这个Notebook进行注册。
按下一个按钮，你就会看到上面出现的gamepad小部件。

### Connect gamepad controller to robot motors

Now, even though we've connected our gamepad, we haven't yet attached the controls to our robot!  The first, and most simple control
we want to attach is the motor control.  We'll connect that to the left and right vertical axes using the ``dlink`` function.  The
``dlink`` function, unlike the ``link`` function, allows us to attach a transform between the ``source`` and ``target``.  Because
the controller axes are flipped from what we think is intuitive for the motor control, we'll use a small *lambda* function to
negate the value.

> WARNING: This next cell will move the robot if you touch the gamepad controller axes!

### 将gamepad控制器连接到机器人电机

现在，尽管我们已经连接了游戏板，但我们还没有将控件连接到我们的小车上！第一个，也是最简单的控制，我们要连接的是马达控制。
这里使用 ``dlink`` 函数将其连接到左右垂直轴。这个 ``dlink`` 函数不同于``link`` 函数，它允许我们在 ``source`` 和 ``target``之间附加一个转换。
因为控制器轴从我们认为对电机控制直观的地方翻转过来，我们将使用一个小的 *lambda* 函数来倒转正负值。

>警告：如果您触摸游戏板控制器轴，下一个单元将移小车！

In [ ]:
from jetbot import Robot
import traitlets

robot = Robot()

left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)

令人惊叹的！小车现在应该对gamepad控制器的动作做出响应。现在从摄像机上观看视频直播！

### 创建和显示图像小部件

首先，让我们显示一个 ``Image`` 小部件，使用它来显示实时相机提要。将 ``hight`` 和 ``width`` 都设置为 300，不会占用太多空间。
>仅供参考：高度和宽度仅影响浏览器端的渲染，而不影响从robot到浏览器的网络传输之前的本机图像分辨率。

In [ ]:
image = widgets.Image(format='jpeg', width=300, height=300)
display(image)

### 创建摄影机实例
好吧，现在没有图像显示，因为我们还没有设置值！只要创建 ``Camera``类并将 ``value`` 属性附加到图像的“value”属性即可！

首先，创建camera实例，我们调用 ``instance`` 方法来创建一个新的 camera，如果它还没有被创造出来。如果已经存在，此方法将返回现有的相机。

In [ ]:
from jetbot import Camera
camera = Camera.instance()

### 将相机连接到图像小部件
Camera类目前只生成BGR8（蓝色、绿色、红色、8bit）格式的值，而图像小部件接受压缩的 *JPEG* 格式的值。
要将相机连接到图像，我们需要在链接中插入 ``bgr8-To-jpeg`` 函数作为转换，如下面步骤

In [ ]:
from jetbot import bgr8_to_jpeg
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

现在您应该看到上面显示的实时视频源了！
> 提醒：可以右键单击单元格的输出，然后选择``Create New View for Output``创建新视图”，将单元格显示在单独的窗口中。

### 如果网络断开，则停止robot

你可以通过视频来驱动你的小车。但如果你的小车与Wifi断开连接呢？好吧，马达会继续移动，它会继续尝试传输视频和马达指令。

让我们做点事，以便在发生断开WIFI连接时停止小车并断开摄像机和电机的连接。

In [ ]:
from jetbot import Heartbeat

def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        left_link.unlink()
        right_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)
# 将回调函数附加到心跳状态
heartbeat.observe(handle_heartbeat_status, names='status')

如果小车与WIFI断开连接，你会注意到它停止了。然后，您可以通过重新创建与下面单元格的链接来重新连接相机和马达

In [ ]:
# 只有当你的机器人链接未链接时才调用这个，否则我们将有多余的链接，这将使传输的命令加倍

left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

### 使用gamepad按钮保存快照
现在，我们希望能够保存一些来自我们机器人的图像。让我们这样做，以便右保险杠（索引5）保存当前实时图像的快照。我们将把这些图像保存在 ``snapshots/`` 目录中，使用 ``uuid`` python包保证名称是唯一的。我们使用 ``uuid1`` 标识符，因为它还对日期和MAC地址进行了编码，我们以后可能会用到它。

In [ ]:
import uuid
import subprocess

subprocess.call(['mkdir', '-p', 'snapshots'])

snapshot_image = widgets.Image(format='jpeg', width=300, height=300)

def save_snapshot(change):
    # save snapshot when button is pressed down
    if change['new']:
        file_path = 'snapshots/' + str(uuid.uuid1()) + '.jpg'
        
        # write snapshot to file (we use image value instead of camera because it's already in JPEG format)
        with open(file_path, 'wb') as f:
            f.write(image.value)
            
        # display snapshot that was saved
        snapshot_image.value = image.value


controller.buttons[5].observe(save_snapshot, names='value')

display(widgets.HBox([image, snapshot_image]))
display(controller)

### Conclusion

That's it for this example, have fun!